In [ ]:
import requests
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Proj, transform
from shapely.geometry import shape, Point

# Setup JSONBin.io
JSONBIN_ACCESS_KEY = '$2a$10$J23yWei2a5JsxL9JkiWOXuZsgZ.qR/GRj74Jre.4i/Te3XjaT0A2y'
JSONBIN_API_URL = 'https://api.jsonbin.io/v3/b/66db5907acd3cb34a87f7d42'

# ArcGIS Polygon Layer GeoJSON URL
ARCGIS_GEOJSON_URL = "https://services1.arcgis.com/7SazdyBTbp9w1Fga/arcgis/rest/services/PlanRVA_admin_boundaries/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Define projections
wgs84 = Proj(init='epsg:4326')  # Geographic (latitude/longitude)
web_mercator = Proj(init='epsg:3857')  # Web Mercator

# Fetch data from JSONBin.io
def get_map_data():
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch point data")
    
# Fetch ArcGIS GeoJSON data
def get_polygon_data():
    response = requests.get(ARCGIS_GEOJSON_URL)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch polygon data")

# Fetch data
point_data = get_map_data()
polygon_data = get_polygon_data()

# Extract features from GeoJSON
point_features = point_data.get('record', {}).get('features', [])
polygon_features = polygon_data.get('features', [])

# Parse points
points = [
    {
        'point': Point(feature['geometry']['coordinates']),
        'name': feature['properties'].get('report_type', 'Unknown')
    }
    for feature in point_features
]

# Parse polygons
polygons = [
    {
        'polygon': shape(feature['geometry']),
        'name': feature['properties'].get('NAME', 'Unknown')  # Adjust key as per GeoJSON properties
    }
    for feature in polygon_features
]

# Summarize points in each polygon
summary = []
for polygon in polygons:
    count = sum(1 for p in points if polygon['polygon'].contains(p['point']))
    summary.append({'polygon_name': polygon['name'], 'point_count': count})

print("Point Summary per Polygon:", summary)

# Convert polygons to Web Mercator for Bokeh
polygon_xs = []
polygon_ys = []
for polygon in polygons:
    x, y = zip(*[transform(wgs84, web_mercator, *coord) for coord in polygon['polygon'].exterior.coords])
    polygon_xs.append(x)
    polygon_ys.append(y)

# Convert points to Web Mercator for Bokeh
point_lons = [transform(wgs84, web_mercator, p['point'].x, p['point'].y)[0] for p in points]
point_lats = [transform(wgs84, web_mercator, p['point'].x, p['point'].y)[1] for p in points]
point_names = [p['name'] for p in points]

# Prepare Bokeh data sources
polygon_source = ColumnDataSource(data=dict(
    xs=polygon_xs,
    ys=polygon_ys,
    name=[p['name'] for p in polygons]
))

point_source = ColumnDataSource(data=dict(
    x=point_lons,
    y=point_lats,
    name=point_names
))

# Set up Bokeh figure
p = figure(
    title="Map with Points and Polygons",
    x_axis_type="mercator", y_axis_type="mercator",
    tools="pan,wheel_zoom,reset,hover"
)
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

# Add polygons
p.patches('xs', 'ys', source=polygon_source, fill_alpha=0.4, line_color="black", line_width=0.5)

# Add points
p.circle(x="x", y="y", source=point_source, size=10, color="blue", alpha=0.6)

# Add tooltips
hover = HoverTool()
hover.tooltips = [
    ("Polygon Name", "@name"),
    ("Longitude", "@x"),
    ("Latitude", "@y")
]
p.add_tools(hover)

# Show map
show(p)

c:\Users\egreenwell\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\egreenwell\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Point Summary per Polygon: [{'polygon_name': 'Powhatan', 'point_count': 1}, {'polygon_name': 'Henrico', 'point_count': 6}, {'polygon_name': 'Chesterfield', 'point_count': 1}, {'polygon_name': 'Richmond', 'point_count': 25}, {'polygon_name': 'Hanover', 'point_count': 1}, {'polygon_name': 'New Kent', 'point_count': 0}, {'polygon_name': 'Charles City', 'point_count': 1}, {'polygon_name': 'Goochland', 'point_count': 0}]


C:\Users\egreenwell\AppData\Local\Temp\ipykernel_70880\106037296.py:76: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = zip(*[transform(wgs84, web_mercator, *coord) for coord in polygon['polygon'].exterior.coords])
